------------------------------------------------------------------------

# TODOs:

-   [ ] Output für OLAT (?)
-   [ ] Feedback einsammeln

------------------------------------------------------------------------

# Von Null auf KI: DIY-Tutorial

    Ein eigenes künstliches neuronales Netz für die Erkennung der Sprache eines
    Textes von Null auf erstellen und trainieren.

## Lernziel

In diesem Tutorial wollen wir eine Beispiel-KI Anwendung in Form eines
`Jupyter-Notebooks` erstellen. Anhand eines gelabelten Datensatzes wird
ein `künstliches neuronales Netz` (KNN) erstellt und trainiert
(`Überwachtes Lernen`). Als Programmiersprache kommt `Python` zum
Einsatz, als KI-Framework nutzen wir
[PyTorch](https://pytorch.org/get-started/locally/).

## Voraussetzungen

-   Eine lauffähige Python/Jupyter Installation
-   (initial) Internetzugriff zum Download der Trainingsdaten
-   Grundlegende Kenntnisse der Programmierung in Python
-   Basiswissen über Neuronale Netze (z.B. aus den anderen OLAT-Kursen
    dieser Reihe)

## Vermittelte Konzepte

-   Prozessschritte bei der Entwicklung von KNNs
    -   Benutzung
    -   Erstellung
    -   Training
    -   Validierung
-   Datenvorbereitung
    -   Datenquellen und Laden von Daten
    -   Datensplit (Training, Validierung, Test)
    -   Datentransformation (Encoding, Decoding) und Embeddings
-   Aufbau des KNN
    -   Schichten (Input, Hidden, Output)
    -   Aktivierungsfunktionen
    -   Gewichte und Biases als Parameter
-   Training
    -   Inferenz (Forward-Pass)
    -   Loss-Function
    -   Stochastic Gradient Descend (Optimizer)
    -   Backpropagation als Lernschritt
    -   Hyperparameter
-   Validierung + Test
-   Anwendung

------------------------------------------------------------------------

# Die Beispielanwendung

In diesem Beispiel wollen wir eine KI erstellen und trainieren, die für
einen Eingabetext die Sprache dieses Textes ermittelt.

## Idee und Herangehensweise

Wir benötigen hierfür eine Datenbasis von bereits `gelabelten Daten`,
also für unser Beispiel eine Sammlung von Texten, denen bereits die
korrekte Sprache zugeordnet ist. Auf der Plattform
[Huggingface](https://huggingface.co) kann man für viele
Problemstellungen Datensammlungen finden, kategorisiert nach
Anwendungsfall.

Wir benutzen in diesem Beispiel die Datensammlung
[language-identification](https://huggingface.co/datasets/papluca/language-identification)
von Huggingsface, welche 70.000 Trainingssätze enthält, wobei jeder Satz
aus einem kurzen Text und dem dazugehörigen Sprachkürzel besteht. Die
Details zu diesem Datensatz lassen sich bei Huggingface aus der
[Dataset-Card](https://huggingface.co/datasets/papluca/language-identification)
entnehmen.

Die Idee für die Spracherkennung ist nun, dass das KNN keine kompletten
Wörter lernt, sondern man die Sprache anhand sogenannter `n-Gramme`
erkennen kann. `n-Gramme` sind Zeichenketten der Länge $n$, ie. ein
`Trigramm` besteht also aus genau $3$ Zeichen.

> Beispiel:  
> Das Wort "Baum" enthält die Trigramme `["Bau", "aum"]`.

Entsprechend werden wir die Texte der Datenbasis jeweils in eine Menge
von Trigrammen umwandeln und diese Mengen dann den Sprach-Labels
zuordnen. Somit reduzieren wir das Vokabular, das das KNN lernen muss
von der Menge aller Wörter einer Sprache auf die in dieser Sprache
vorkommenden Trigramme.

Das Training erfogt nun anhand dieser Trigramm-Mengen. Nach Abschluss
soll das KNN nun für eine gegebenen Trigramm-Menge die wahrscheinlichste
Sprache liefern.

<figure>
<img src="./img/DIY-Tutorial-Usage.drawio.svg"
alt="Ablauf für unseren Use-Case" />
<figcaption aria-hidden="true">Ablauf für unseren Use-Case</figcaption>
</figure>

------------------------------------------------------------------------

# Theorieteil: Prozessschritte für das Arbeiten mit KNN

Um einen besseren Überblick über die einzelnen Schritte in diesem
Tutorial zu geben folgt hier nun ein kurzer Überblick über die
verschiedenen Prozessschritte beim Arbeiten mit KNNs.

## Benutzen von KNNs

<figure>
<img src="img/Processes-Usage.svg"
alt="Allgemeine Prozesskette bei der Nutzung von KNNs als Funktion" />
<figcaption aria-hidden="true">Allgemeine Prozesskette bei der Nutzung
von KNNs als Funktion</figcaption>
</figure>

In der Nutzung funktioniert ein KNN wie eine reguläre Funktion: Es
berechnet zu einer Eingabe eine Ausgabe. Da KNNs mit
[Tensoren](https://pytorch.org/docs/stable/tensors.html) arbeiten,
müssen Ein- und Ausgabedaten entsprechend transformiert werden
(`Encoding`, `Decoding`). Im Interpretationsschritt wird dann aus der
Ausgabe des Netzes eine Aussage gewonnen. Für die Klassifikation gibt
ein KNN typischerweise die Wahrscheinlichkeit für jede Klasse aus, dass
die Eingabe zu dieser Klasse gehört. Ein trivialer
Interpretationsschritt ist dann also, die Klasse mit der höchsten
Wahrscheinlichkeit als Ergebnis zu nehmen.

## Erstellen eines KNNs

Möchte man ein KNN selbst erstellen so legt man zum einen die Form der
Ein- und Ausgabe fest, zum anderen aber auch die Topologie und die
Aktivierungsfunktionen.

<figure>
<img src="img/Processes-Construction.svg"
alt="Prozessschritte beim Erstellen eines KNNs" />
<figcaption aria-hidden="true">Prozessschritte beim Erstellen eines
KNNs</figcaption>
</figure>

## Training eines KNNs

Der Erfolg beim Training eines KNNs liegt neben der technischen
Vorgehensweise vor allem auch an der Qualität der Trainingsdaten. Eine
Betrachtung der Kriterien für "gute" Trainingsdaten würde den Rahmen
dieses Tutorials sprengen. Wir benutzen daher, wie üblich in der
wissenschaftlichen Methodik, bereits bestehende und im Idealfall nach
guten Standards zusammengestellte Daten. Dies sorgt zudem für eine
bessere Reproduzierbarkeit unsere Ergebnisse. Die Datenerstellung
entfällt somit in unserem Beispiel, ist aber einer der wichtigsten
Prozessschritte.

Beim hier betrachteten überwachten Lernen (`Supervised Learning`)
unterteilt man die verfügbaren Datensätze in eine Trainingsmenge,
Validierungsmenge und Testmenge. Nur die Trainingsmenge wird für das
Training benutzt, so dass die Daten aus der Validierungs- und Testmenge
für das Netz unbekannt bleiben. So lässt sich später prüfen, ob das Netz
die Trainingsdaten lediglich auswendig gelernt hat (`Overfitting`), oder
ob es die gelernten Zusammenhänge auch korrekt auf neue Daten übertragen
kann.

Bei Training eines KNNs wird also die Menge an Trainingsdaten durch das
Netz propagiert (`Forward Pass`, `Inferenz`) und dabei jeweils die
Abweichung zwischen ausgegebenem und erwartetem Ergebnis berechnet
(`Loss`). Anhand dieser Abweichung werden die Parameter des Netzes
(`Weights`, `Biases`) angepasst.

Die Art der Anpassung bestimmt der `Optimizer` anhand der partiellen
`Gradienten` der Netzkomponenten und der Lernrate (`Learning Rate`).
Dies geschieht in vielen Iterationen/Epochen (`Epochs`) bis eine
gewünschte minimale Abweichung erreicht worden ist. An dieser Stelle ist
wichtig zu erwähnen, dass dieser Prozess immens rechenintensiv ist und
deshalb verschiedene Methoden entwickelt wurden, um den Rechenaufwand zu
verringern. Eine dieser Methoden ist das sogenannte `Batching`. Hierbei
wird in jeder Lerniteration nur ein jeweils zufällig ausgewählter Teil
(`Batch`) der Trainingsdaten durch das Netz propagiert. Die Größe dieser
Batches ist ein weiterer `Hyperparameter`, neben `Learning Rate` und
`Epochs`.

<figure>
<img src="img/Processes-Training.svg"
alt="Training eines KNNs" />
<figcaption aria-hidden="true">Training eines KNNs</figcaption>
</figure>

In der KI-Community hat sich der Begriff `Hyperparameter` für die
Konfigurationsgrößen des Lernprozesses etabliert, `Parameter` hingegen
bezeichnen die Koeffizienten (`Weights` und `Biases`) der Netzschichten
an sich.

Als Merkregel lässt sich ungefähr angeben: `Parameter` werden durch das
Training verändert, `Hyperparameter` durch den Durchführenden.

## Validierung eines KNNs

Um die Qualität des Netzes zu bestimmen, wird während des
Trainingsprozesses die Genauigkeit der Ergebnisse (`Accuracy`) anhand
dem Netz noch unbekannter Daten (`Validation Data`) überprüft. Stellt
man fest, dass die aktuelle Trainingskonfiguration nicht zum Erfolg
führt, so passt man die Trainingsparameter (`Hyperparameter`) wie
Lernrate, Anzahl der Lernschritte (`Epochs`) oder den Optimizer an, in
manchen Fällen auch die Topologie des Netzes.

<figure>
<img src="img/Processes-Validation.svg"
alt="Validierung eines KNNs" />
<figcaption aria-hidden="true">Validierung eines KNNs</figcaption>
</figure>

## Testen eines KNNs

Das Testen erfolgt analog zur Validierung, jedoch wird hier der
Testdatensatz benutzt. Hier wird ebenfalls die `Accuracy` des Models
anhand der korrekt und fehlerhaft klassifizierten Texte ermittelt.
Während die Validierung die Aufgabe hat, die Trainingsfortschritte zu
bewerten, werden beim Testen die Verbesserungen durch die Änderungen der
Hyperparameter gemessen. Aus diesem Grund werden auch zwei verschiedene
zusätzliche Datasets benötigt. In unserem Beispiel verzichten wir im
weiteren auf die äußerste (manuelle) Iterationsschleife über
verschiedene Hyperparameter und benötigen somit auch kein explizites
Testing.

## Gesamtprozessbild

Zur Übersicht hier das für unser aktuelles Beispiel relevante
Prozessschaubild. Die gelben Rechtecke stellen Entscheidungen dar, die
zu treffen sind, die weißen Rechtecke statische Prozessschritte. `KNN`
bezeichnet das Netz, `LF` die *Loss-Function* und `Opt` den *Optimizer*.

<figure>
<img src="img/Processes.drawio.svg"
alt="Gesamtprozessbild" />
<figcaption aria-hidden="true">Gesamtprozessbild</figcaption>
</figure>

------------------------------------------------------------------------

# Praxisteil: Implementierung in Python mit PyTorch

## Arbeitumgebung

Auf die Einrichtung einer Python- und Jupyter-Umgebung wird in diesem
Tutorial nicht gesondert eingegangen. Vorausgesetzt wird:

-   Python \>=3.10
-   Python-Packages:
    -   pandas \>=2.1.2
    -   torch \>=2.1.1
    -   tqdm \>=4.66.1
    -   tensorboard \>=2.15.0

Diese lassen sich je nach Paketmanager beispielsweise so installieren:

In [ ]:
!pip install pandas torch tqdm tensorboard

In [ ]:
!conda install pandas torch tqdm tensorboard

------------------------------------------------------------------------

## Vorbereitung der Daten

### Datenauswahl / -erstellung

Dieser Schritt entfällt in unserem Beispiel, da wir ein bereits
vorbereitetes Dataset benutzen wollen.

### Herunterladen und Split der Daten

Im ersten Schritt laden wir die 3 Datensätze (Training, Test,
Validierung) lokal von Huggingface herunter. Diese liegen dann im Ordner
`data/demo/`.

In [ ]:
import os
import urllib

def cacheFile(file: str, url: str) -> str:
    """Downloading a file, if it does not yet exist locally"""
    if not os.path.exists(file):
        path = os.path.dirname(file)
        os.makedirs(path, exist_ok=True)

        def progress(num, size, total):
            """Prints download progress"""
            perc = round(num * size / total * 100)
            print(f"Downloading {file}: {perc}%", end="\r")

        urllib.request.urlretrieve(url, file, progress)
    else:
        print(f"File already downloaded: {file}")
    return file

file_train = cacheFile("data/demo/train.csv", "https://huggingface.co/datasets/papluca/language-identification/resolve/main/train.csv?download=true")
file_valid = cacheFile("data/demo/valid.csv", "https://huggingface.co/datasets/papluca/language-identification/resolve/main/valid.csv?download=true")
file_test = cacheFile("data/demo/test.csv", "https://huggingface.co/datasets/papluca/language-identification/resolve/main/test.csv?download=true")

### Laden und erste Sichtung der Daten

Im nächsten Schritt laden wir die Daten in einen
[pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html),
also in eine Hilfsstruktur, die die Daten für die einfachere
Transformation speichert.

Der nächste Code-Block zeigt das Einlesen der zuvor heruntergeladenen
Trainingsdaten und zeigt die ersten Einträge exemplarisch an.

Wir sehen eine Tabelle, bestehend aus Sprachlabels in der Spalte `label`
und den dazugehörigen Text in der Spalte `text`.

       labels text
    0     pt  os chefes de defesa da estónia, letónia, lituâ...
    1     bg  размерът на хоризонталната мрежа може да бъде ...
    2     zh  很好，以前从不去评价，不知道浪费了多少积分，现在知道积分可以换钱，就要好好评价了，后来我就把...
    3     th  สำหรับ ของเก่า ที่ จริงจัง ลอง   honeychurch  ...
    4     ru                             Он увеличил давление .

Als weitere Ausgabe sehen wir die Anzahl Datensätze pro Sprache, in
diesem Fall je 3500.

    labels
    pt    3500
    bg    3500
    en    3500
    vi    3500
    ...

In [ ]:
import pandas as pd

df_train = pd.read_csv(file_train)
df_valid = pd.read_csv(file_valid)

print(df_train.head())
print()
print(df_train["labels"].value_counts())

------------------------------------------------------------------------

## Datentransformation

Wie in der Einleitung besprochen, wollen wir beim Training des KNNs
nicht komplette Wörter aus jeder Sprache lernen, sondern Texte vorher in
Trigramm-Mengen umwandeln und anhand dieser die Sprache identifizieren.

Für eine später einfachere Anpassung codieren wir die Transformation
aber für beliebig lange Teilworte: `n_Gramme`.

### n-Gramme

Wir erstellen eine Hilfsfunktion, die aus einem Text die Menge der
enthaltenen n-Gramme erzeugt.

Für die Beispieltexte (siehe Code) erhalten wir die zwei Ausgabemengen:

    {'aum', 'Bau'}

    {' ei', ' De', 'Men', '...', 'ne ', ' ..', ' is', 'ist', ' Me', 'on ', 'lan', 's i', 'n .', 'ine', ' de', 'eng', ' gr', 'öße', 'röß', 'es ', 'grö', ' vo', 'e M', 'ext', 'xt,', 'Die', 'Dem', 'ßer', 't e', 'st ', 'ge ', ', d', 'von', 'n l', ' la', 'ies', 'er ', 'r e', 'e g', 'ang', 'emo', 'r D', 'in ', 'e v', 'ein', 'ere', 're ', 'mo-', 'o-T', 'der', 'ger', '-Te', 't, ', 'Tex', 'nge'}

In [ ]:
def nGrams(text, size = 3):
    """Returns a set of unique n-grams for a given text"""
    result = set()
    # pairwise iterator over text
    for start, end in zip(range(0, 1 + len(text) - size), range(size, 1 + len(text))):
        result.add(text[start:end])
    return result

print(nGrams("Baum"))
print(nGrams("Dies ist ein langer Demo-Text, der eine größere Menge von ..."))

### Encodings und Embeddings

Um nun diese Daten für das Training eine KNN zu nutzen, müssen wir noch
definieren, wie die Eingabe und die Ausgabe des KNN codiert sein soll.
Für Klassifikation von (kurzen) Texten bietet sich das sogenannte
`Hot-Encoding` an. Hierbei codiert ein Vektor, dessen Komponenten `0`
oder `1` sind, welche Werte aus einer Grundmenge gesetzt, also `hot`
sind. Ein Beispiel:

Angenommen wir haben die Grundmenge der Trigramme gegeben als Liste:

> Liste $T$ der Trigramme: \["Alm", "aum", "Bau", "ber", ..., "XYZ"\],  
> Anzahl Elemente in $T$: $n = |T|$

, so ergibt sich der `Hot-Encoded-Vektor` als Vektor der Länge $n$
(=Anzahl der unterschiedlichen Trigramme), bei dem jeweils die
Komponenten `1` gesetzt sind, die in der Trimgramm-Menge des Textes
"Baum" vorkommen, also:

> Hot-Encoding für "Baum": \[0, 1, 1, 0, ..., 0\].

In der Literatur werden `Encoding` und `Embedding` häufig synonym
verwendet, wobei
``` Encoding`` auf die technische Konvertierung abzielt (diese ist nicht unbedingt semantikerhaltend), und ```Embedding\`\`
die Codierung in eine semantikerhaltende Domäne meint bzw. auch die
Repräsentation des Datenpunktes in dieser Domäne. Semantikerhaltend
bedeutet in diesem Zusammenhang, dass relevante Eigenschaften durch die
Transformation nicht verloren gehen. Als Beispiel:

Bei der Modellierung von
[Large-Language-Models](https://en.wikipedia.org/wiki/Large_language_model)
ist die Bedeutung von Worten wichtig, ein Embedding sollte also
sicherstellen, dass ähnliche Bedeutungen bei der Codierung als Tensoren
"nahe beieinander" liegen bzw. sogar auf denselben Feature-Vektor
abgebildet werden, wodurch bereits der erste Abstraktionsschritt für das
Lernen erfolgt.

In unserem Fall ist nicht die Beibehaltung der Bedeutung der Worte
wichtig, sondern lediglich die Beibehaltung desselben Alphabets und der
Reihenfolge von Buchstaben um die Sprache eines Textes zu
identifizieren. Ein `Hot-Encoding` von Trigrammen erfüllt diese
Anforderung und ist entsprechen ein passendes `Embedding`.

### Erstellen der Input-Tensoren

Beginnen wir mit der Erstellung der Trigramm-Grundmenge. Hierbei
berechnen wir für alle Einträge des Trainingsdatensatzes die
Trigramm-Mengen und vereinigen diese. Da wir hier mit Mengen arbeiten,
entstehen keine doppelten Einträge. Somit erhalten wir das komplette
Vokabular aller Trigramme aller Sprachen.

Da wird später für jedes Trigramm einen Index benötigen, legen wir die
Trigramme in einem Dictionary ab, welches jedem Trigramm einen
eindeutigen Index zuordnet. Diese Art der Datenstruktur nennt man auch
`Lookup-Table`. Dieser hilft bei der effizienten Erstellung der
`Hot-Encoded-Vektoren` als `Embedding`.

Es entstehen 346740 verschiedene Trigramme (siehe Code):

    Count:  346740
    Peek:  ['   ', '  "', '  $', '  %', '  &', "  '", ...
    Lookup:  [('   ', 0), ('  "', 1), ('  $', 2), ('  %', 3), ...

In [ ]:
def makeTrigrams():
    trigrams = sorted(list(set().union(*df_train['text'].map(lambda t: nGrams(t)))))
    trigramsLookup = dict(zip(trigrams, range(len(trigrams))))
    return trigrams, trigramsLookup

trigrams, trigramsLookup = makeTrigrams()
print("Count: ", len(trigrams))
print("Peek: ", trigrams[0:20])
print("Lookup: ", list(trigramsLookup.items())[0:20])

Als nächstes benötigen wir nun eine Funktion, um aus einem Text den
finalen Embedding-Vektor zu erzeugen. Da KNNs mit Tensoren arbeiten,
erzeugen wir den Vektor in Tensor-Form. Für das effiziente Arbeiten mit
Tensoren, kann Pytorch auf verschiedener Hardware rechnen, wie
[CUDA](https://de.wikipedia.org/wiki/CUDA),
[MPS](https://de.wikipedia.org/wiki/Metal_(API)#Metal_Performance_Shader)
oder auf der CPU. Deshalb folgt zuerst eine kurze Methode, die die
verfügbare Hardware auswählt.

In [ ]:
import torch

deviceName = "cuda:0" if torch.backends.cuda.is_built() \
    else "mps" if torch.backends.mps.is_built() \
    else "cpu"
device = torch.device(deviceName)
torch.set_default_device(device)

print("Using device: " + str(device))

Für unser Beispiel "Baum" erhalten wir folgenden `Hot-Tensor`, mit der
Länge 346740 und den Hot-Indizes an den Stellen \[24552, 35275\].

    Hot-Embedding:
        Tensor-Size:  346740
        Tensor:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0')
        Hot-Indices:  [24554, 35275]

In [ ]:
import torch

def makeHotTensor(text, _trigrams=trigramsLookup):
    tensor = torch.zeros(1, len(_trigrams))
    for trigram in nGrams(text):
        idx = _trigrams.get(trigram)
        if idx != None:
            tensor[0][idx] = 1
    return tensor

hotExample = makeHotTensor("Baum")
hotIndices = ((hotExample[0] == 1).nonzero(as_tuple=True)[0])
print("Hot-Encoding:")
print("\tTensor-Size: ", hotExample.size(dim=1))
print("\tTensor: ", hotExample)
print("\tHot-Indices: ", hotIndices.tolist())

### Erstellen der Output-Tensoren

Bei unserer Beispielaufgabe handelt es sich um eine Klassifizierung. Für
die Codierung der Inputs haben wir eine Abbildung für Texte in
Hot-Tensoren erstellt.

Für die Codierung der Outputs gehen wir analog vor: Wir codieren die
möglichen Sprachen ebenfalls als Hot-Tensoren. Da die Ausgabe immer
genau eine Sprache liefern soll, benutzen wir ein sog.
`One-Hot-Encoding`, also ähnlich wie bei der Eingabe, nur dass bei den
Ausgabe-Tensoren immer genau nur eine Komponente `1` ist.

Zuerst bilden wir die Menge aller Sprachen und konvertieren diese in
eine Lookup-Struktur um eine Reihenfolge zu definieren. Wir erhalten:

    Count:  20
    Peek:  {'ar': 0, 'bg': 1, 'de': 2, 'el': 3, 'en': 4, 'es': 5, 'fr': 6, 'hi': 7, 'it': 8, 'ja': 9, 'nl': 10, 'pl': 11, 'pt': 12, 'ru': 13, 'sw': 14, 'th': 15, 'tr': 16, 'ur': 17, 'vi': 18, 'zh': 19}

In [ ]:
def makeLanguages():
    languages = sorted(list(set(df_train['labels'])))
    languagesLookup = dict(zip(languages, range(len(languages))))
    return languages, languagesLookup

languages , languagesLookup = makeLanguages()
print("Count: " , len(languages))
print("Peek: ", languages)

Schließlich benötigen wir eine Funktion, um für eine Sprache den
Embedding-Tensor zu erzeugen. Für die Sprache Deutsch erhalten wir
beispielsweise einen Tensor der Länge 20 (=Anzahl der Sprachen), bei dem
an Index 2 der Wert 1 gesetzt ist:

    One-Hot-Encoding:
        Tensor-Size:  20
        Tensor:  tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0.]], device='mps:0')
        Hot-Indices:  [2]

In [ ]:
def makeOneHotTensor(lang, _languages=languagesLookup):
    tensor = torch.zeros(1, len(_languages))
    idx = _languages.get(lang)
    if idx != None:
        tensor[0][idx] = 1
    return tensor

langTensor = makeOneHotTensor("de")
langIndex = ((langTensor[0] == 1).nonzero(as_tuple=True)[0])
print("One-Hot-Encoding:")
print("\tTensor-Size: ", langTensor.size(dim=1))
print("\tTensor: ", langTensor)
print("\tHot-Indices: ", langIndex.tolist())

### Zusammenfassung

Damit ist die Datenvorbereitung abgeschlossen. Hier noch einmal als
Zusammenfassung alle Schritte:

-   Datenbeschaffung und Aufteilen in 3 Mengen: Training, Validierung,
    Test
-   Transformation der Texte als Trigramm-Menge und Embedding:
    Hot-Tensoren
-   Transformation der Sprachlabels als Embedding: Hier als
    One-Hot-Tensoren

------------------------------------------------------------------------

## Erstellen eines KNN

### Netztopologie

Bei der Erstellung eines KNNs sind zuerst die `Input`- und
`Output-Layers` zu definieren. Diese ergeben sich auf der Codierung der
Eingabe- und Ausgabedaten. Dazwischen befinden sich ein oder mehrere
sog. `Hidden-Layers`. Je nach Literatur und Framework wird die
Zählweise, was erster und letzter Hidden-Layer sind unterschiedlich
behandelt, abhängig davon, ob Input- und Output-Layer bereits
Berechnungslayer sind oder die Werte nur durchreichen.

In Pytorch orientiert sich das Layer-Konzept an den
Berechnungsschritten, so dass hier eine logische Neuronen-Schicht in
ein- oder mehrere PyTorch-Layer aufgeteilt ist, da

-   die Lineartransformation der Inputs mit den Gewichten und optional
    Bias:  
    $y' = x A^T + b$, mit:  
    $y'$: Output-Tensor, $x$: Input-Tensor, $A^T$: Gewichtstensor, $b$:
    Bias-Tensor
-   und die (nichtlineare) Aktivierungsfunktion: $y = sigmoid(y')$

getrennt angelegt werden.

Für die Implementierung wird von der Basisklasse `nn.Module` abgeleitet.
In `__init__` werden die benötigten PyTorch-Layer angelegt, in der
Methode `forward` dann die Aufrufe zur Berechnungen hintereinander
ausgeführt.

In unserem Fall erstellen wir ein `Feed-Forward-Net` (FNN), also eine
Topologie ohne Schleifen, mit einem Input-Layer inklusive
Linear-Transformation, einem Hidden-Layer mit der nichtlinearen
Aktivierungsfunktion und einem Output-Layer mit Transformation aber ohne
Aktivierungsfunktion.

Diese Festlegungen sind erst einmal willkürlich. Es hat sich aber als
sinnvoll erwiesen mit einfachen Topologien zu beginnen und erst bei
ausbleibendem Lernerfolg auf komplexere Topologien zu wechseln.

Die Größe der Layer, ie. Anzahl der künstlichen Neuronen je Layer geben
wir als Parameter der `__init__`-Funktion an.

Zuletzt erstellen wir eine Instanz des Netzes und geben die Stuktur
textuell aus:

    FeedForwardNet(
    (input_layer): Linear(in_features=346740, out_features=40, bias=True)
    (input_activation): Sigmoid()
    (output_layer): Linear(in_features=40, out_features=20, bias=True)
    )

In [ ]:
import torch.nn as nn

class FeedForwardNet(nn.Module):
   def __init__(self, input_size, hidden_size, output_size):
       super(FeedForwardNet, self).__init__()

       self.input_layer = nn.Linear(input_size, hidden_size)
       self.hidden_layer = nn.Sigmoid()
       self.output_layer = nn.Linear(hidden_size, output_size)

   def forward(self, x):
       y_1 = self.input_layer(x)        # linear
       y = self.hidden_layer(y_1)       # sigmoid
       out = self.output_layer(y)       # linear
       return out

input_size = len(trigrams)
output_size = len(languages)
hidden_size = 40

model = FeedForwardNet(input_size, hidden_size, output_size)

print(model)

------------------------------------------------------------------------

## Training

### Definition der Hyperparameter

Wie im Abschnitt über die Prozessschritte angesprochen folgen nun die
Festlegungen für die eigentliche Trainings- und Validierungsroutinen.

Auch hier gilt: die Entscheidungen für die einzelnen Settings sind
initial recht willkürlich gewählt und werden später optional angepasst,
wenn das Training nicht erfolgreich verläuft.

Wir wählen als Loss-Function
[Cross-Entropie-Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html),
dies ist für unser Klassifikationsproblem und die Codierung als
Hot-Vektoren gut geeignet.

Die Learning-Rate legen wir mit $0.05$ fest. Dies beeinflusst wie stark
die Anpassung der Gewichte in Richtung der Gradienten in jeder
Lerniteration erfolgt. Hierbei gilt: Bei kleineren Lernraten dauert das
Training länger, aber die Wahrscheinlichkeit von Oszillation und
Instabilitäten ist größer.

Für die Optimierung wählen wir als Verfahren [Stochastic Gradient
Descent](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)
(SDG) und übergeben die Model-Parameter und die Learning-Rate.

Zusätzlich erstellen wir noch eine Hilfsfunktion, die den Fortschritt
optional detailliert ausgibt. Hierfür nutzen wir die Bibliothek
[tqdm](https://github.com/tqdm/tqdm).

In [ ]:
from tqdm.auto import tqdm
import torch.nn as nn
import torch.optim as optim

lossFunction = nn.CrossEntropyLoss()
learning_rate = 0.05
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

PROGRESS_DETAIL=False

def progress(len, unit, desc):
    iter = range(len)
    return tqdm(iter, unit=unit, desc=desc) if PROGRESS_DETAIL else iter

### Die Trainingsfunktion

Die Trainingsfunktion bekommt als Eingabe einen `Batch` an
Trainingsdaten, wobei jeder Satz die Attribute `text` und `labels`
trägt. Initial wird das Model in den Trainingsmodus geschaltet. Die
Sätze werden dann in einer Schleife jeweils abgearbeitet:

-   Zurücksetzen der Gradienten (dies wird häufig vergessen, ist aber
    wichtig, da in Pytorch Gradienten standardmäßig akkumuliert werden)
-   Input-Text als HotTensor codieren
-   Gelabelte-Sprache als OneHotTensor codieren
-   Output-Tensor durch das Netz berechnen lassen
-   Loss berechnen als Delta zwischen Label und Output
-   Loss rückwärts durch das Netz propagieren (Gradientenberechnung)
-   Gewichte anpassen durch Aufruf des Optimizers
-   Gesamt-Loss für den Batch berechnen

In [ ]:
def train(batch):
    model.train()
    epoch_loss = 0.0
    for i in progress(len(batch), unit="sample", desc="Training"):
        optimizer.zero_grad()                                   # !important
        input = makeHotTensor(batch["text"].iloc[i])
        expected = makeOneHotTensor(batch["labels"].iloc[i])
        output = model(input)
        loss = lossFunction(output, expected)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return loss / len(batch)

### Die Validierungsfunktion

Die Validierung läuft analog zum Training. Auch hier ist die Eingabe ein
Batch, diesmal jedoch nicht aus der Trainingsmenge, sondern aus der
Validierungsmenge. Somit ist sichergestellt, dass das Netz diese Daten
beim Training noch nicht gesehen hat.

Zuerst wird das Modell in den Evaluierungsmodus geschaltet und die
Gradientenberechnung deaktiviert. Dies erhöht die Auswertungsperformance
und ist für die reine Inferenz nicht nötig. Analog zum Training
durchlaufen die Datensätze folgende Schleife:

-   Input-Text als HotTensor codieren
-   Ausgabe des Netzes berechnen
-   Interpretationsschritt: aus dem Ausgabetensor die Sprache mit der
    höchsten Wahrscheinkeit wählen via `torch.argmax`
-   Variablen mit der Gesamtzahl und der Anzahl korrekter Predictions
    aktualisieren

In [ ]:
def evaluate(batch):
    model.eval()
    total = 0.0
    correct = 0.0
    with torch.no_grad():
        for i in progress(len(batch), unit="sample", desc="Evaluating"):
            input = makeHotTensor(batch["text"].iloc[i])
            output = model(input)
            predicted = languages[torch.argmax(output)]
            total += 1.0
            expected = batch["labels"].iloc[i]
            correct += predicted == expected
    return total, correct

### Die Trainingsiterationen

Das eigentliche Training wird durch eine äußere Schleife realisiert.
Hier definieren wir die `Hyperparameter` für die Anzahl Lern-Epochen und
die Batchgröße. Zusätzlich schreiben wir eine Hilfsfunktion, die den
Lernfortschritt ausgibt. Die Genauigkeit des Modells (`Accuracy`)
erhalten wir aus dem Ergebnis der `Validierung`, den aktuellen Loss-Wert
aus dem `Training`.

Die Ausführung des nächsten Codeblocks führt das eigentliche Training
durch. Zu beachten ist, das in Jupyter-Notebooks die Variablen global
sind und erst bei Neuauswertung einer Zelle neu gesetzt werden. Möchte
man also ein neues Training starten, so sollte man oben die Zelle zur
Erstellung des Modells neu ausführen, damit es neu initialisiert ist.

Für die visuelle Überwachung des Trainingfortschritts setzen wir
zusätzlich zur textuellen Augabe im Notebook das Tool
[Tensorboard](https://pytorch.org/docs/stable/tensorboard.html) ein.
Hierbei werden lokal im Ordner `runs` Log-Dateien während des Trainings
geschrieben, anhand derer eine grafische Visualisierung per Website
erfolgt (Default-Url: http://localhost:6006).

In [ ]:
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

def log(entries):
    [writer.add_scalar(e[0], e[1], e[2]) for e in entries]
    print("\n".join(map(lambda e: f"{e[0]}: {e[1]}", entries)))

EPOCHS = 100
BATCH_SIZE = 50

for epoch in tqdm(range(EPOCHS), unit="epoch"):
    total, correct = evaluate(df_valid.sample(BATCH_SIZE))
    loss = train(df_train.sample(BATCH_SIZE))
    log([
        ["Accuracy/train [%]", correct*100/total, epoch],
        ["Loss/train", loss, epoch]
    ])

## Nutzung des trainierten KNNs

Ist unser Netz nun trainiert, so lässt es sich als simple Funktion
nutzen, wie in folgendem Code-Block gezeigt.

Hierbei werden meist alle Sprachen korrekt erkannt, problematisch sind
beispielsweise Japanisch und Chinesisch. Für diese Sprachen eignet sich
unser Trigramm-Konzept nicht so gut.

In [ ]:
def analyzeLanguage(text, _model = model):
    _model.eval()
    with torch.no_grad():
        input = makeHotTensor(text)
        output = _model(input)
        lang = languages[torch.argmax(output)]
    print(f'{text}: {lang}')
    return lang

analyzeLanguage("Heute ist ein schöner Tag")            # de - Deutsch
analyzeLanguage("Today is a beautiful day")             # en - Englisch
analyzeLanguage("Aujourd'hui est une belle journée")    # fr - Französisch
analyzeLanguage("Bugün güzel bir gün")                  # tr - Türkisch
analyzeLanguage("今日は素晴らしい日です")                   # ja - Japanisch
analyzeLanguage("今天是美好的一天")                       # zh - Chinesisch

### Speichern und Laden trainierter Modelle

Wollen wir unser trainiertes Modell speichern, so stehen verschiedene
Methoden zur
[Auswahl](https://pytorch.org/tutorials/beginner/saving_loading_models.html).
Am besten eignet sich für die Weitergabe Torchscript, da hierbei sowohl
Netztopologie als auch Parameter gespeichert werden.

In [ ]:
import torch

# Export as TorchScript
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('data/model.ts.pt') # Save

# Load model as TorchScript
model2 = torch.jit.load('data/model.ts.pt')
model2.eval()

# Test
langSavedModel = analyzeLanguage("Today is a beautiful day", model)
langLoadedModel = analyzeLanguage("Today is a beautiful day", model2)
print(langSavedModel, langLoadedModel)

------------------------------------------------------------------------

# Zusammenfassung

In diesem Tutorial haben wir gesehen, welche Schritte benötigt werden um
ein KNN zu trainieren:

-   Vorbereitung der Daten
-   Encoding von Inputs und Labels als Hot-Tensoren
-   Netztopologie und Hyperparameter
-   Training und Validierung
-   Trainingsepochen und Batches
-   Loss und Accuracy
-   Inferenz
-   Nutzung des trainierten KNNs
-   Laden und Speichern von Modellen